# RAW EXTRACT

In [ ]:
import datetime
import json
import datetime
try:
    import tweepy
except:
    !pip install tweepy
    import tweepy
import re

In [ ]:
def twitter_extract( twitter_ids, no_of_tweets ):
    
    consumer_key    = 'Sd08uC11SZLI8ruRhSYxe7DVj'
    consumer_secret = '3IzjDnFPCirqHRgNXJkHeNE4u26DJM5bVVDrogWqmUXPmrmTe6'
    access_key      = '897433926054039552-9dCfAeZNMlae5jck5lLh0JO2HS95dSl'
    access_secret   = '3MDWXGwp2AEvyagOIjq0BVsk8kgtKJwTehxPdi1JiOFdA'
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret) 
    auth.set_access_token(access_key, access_secret) 
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)
    
    final_dict = {}
    
    count = 0
    for twitter_id in twitter_ids:
    
        status_cursor    = tweepy.Cursor(
            api.user_timeline, 
            screen_name  = twitter_id,
            count        = no_of_tweets,
            tweet_mode   = 'extended'
        ).items(no_of_tweets)
            
        tweets = {}
        for status in status_cursor:
            date = status.created_at
            date_str = date.strftime("%Y,%m,%d")
            tweets[date_str] = status._json['full_text']
            
        final_dict[twitter_id] = tweets
        
        count = count+1
        print(count,'   ',twitter_id,' extraction done')
    return final_dict      

In [ ]:
# Add Persons Twitter Id in twitter_id_list before execution
twitter_id_list = json.load( open( 'input_files/twitter_ids.json' ,'r' ))

In [ ]:
tweets_raw = twitter_extract(twitter_id_list, 2000)

In [ ]:
import json
#tweets_raw = json.load(open('extracted/tweets_raw.json', 'r', encoding='utf8'))
dates_dict = json.load(open('input_files/country_dates.json', 'r'))

for country in dates_dict.keys():
    if dates_dict[country] != "None":
        dates_dict[country]['start_pre']  = datetime.datetime.strptime(
            dates_dict[country]['start_pre'] ,"%Y,%m,%d")
        dates_dict[country]['end_pre']    = datetime.datetime.strptime(
            dates_dict[country]['end_pre']   ,"%Y,%m,%d")
        dates_dict[country]['start_post'] = datetime.datetime.strptime(
            dates_dict[country]['start_post'],"%Y,%m,%d")
        dates_dict[country]['end_post']   = datetime.datetime.strptime(
            dates_dict[country]['end_post']  ,"%Y,%m,%d")

In [ ]:
tweets_pre_covid   = {}
tweets_post_covid  = {}

for person in tweets_raw.keys():
    
    country = twitter_id_list[person] 
    if( dates_dict[country] == 'None'):
        country = 'default'
    text_pre_dict  = {}
    text_post_dict = {}
    
    for date in tweets_raw[person].keys():
        date_text = datetime.datetime.strptime(date, '%Y,%m,%d')
        if( date_text >= dates_dict[country]['start_pre'] and date_text <= dates_dict[country]['end_pre']  ):
            text_pre_dict[date] = tweets_raw[person][date]
        if( date_text >= dates_dict[country]['start_post'] and date_text <= dates_dict[country]['end_post']  ):
            text_post_dict[date] = tweets_raw[person][date]
    
    if( text_pre_dict and text_post_dict):
        tweets_pre_covid[person]  = text_pre_dict
        tweets_post_covid[person] = text_post_dict
    else:
        print(person)

In [ ]:
with open( 'extracted/tweets_pre_covid.json', 'w') as outfile:
            json.dump(tweets_pre_covid, outfile)

with open( 'extracted/tweets_post_covid.json', 'w') as outfile:
            json.dump(tweets_post_covid, outfile)